<a href="https://colab.research.google.com/github/Johyeonje/Start_Tensorflow_2.0_Programming/blob/master/Start!_chap6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Image

#컨볼루션 신경망

: 특징 추출의 기법중 하나로 원본 이미지의 각 픽셀을 포함한 주변 픽셀과 필터의 모든 픽셀은 각각 곱연산을 하고 그 결과를 모두 합연산을 해주기 때문에 합성곱 이라고도 부름

이를 네트워크에 접합하여 `네트워크가 특징을 추출하는 필터를 자동으로 생성`하게 하는 것

##주요 레이어 정리

Dense : Fully Connected layer    
Flatten : 다차원의 이미지를 1차원을 평평하게 바꿔주는 단순 레이어

컨볼루션 신경망은 `특징 추출기(Feature Extractor)`와 `분류기(Classifier)`가 합쳐져 있는 형태이다.    
**특징 추출기**의 역할을 하는 것은 **컨볼루션 레이어**와 **풀링 레이어**이고,    
**분류기**의 역할이 **Dense 레이어**이다.

일반적인 컨볼루션 신경망의 구조에서는 **컨볼루션 레이어와 풀링 레이어가 교차**되며 배치되고,    
분류기에는 Dense레이어가 배치되며 과적합 방지를 위해 `드롭아웃` 레이어가 **Dense 레이어 사이**에 배치된다.

컨볼루션 레이어의 값은 학습 중에 자동으로 추출되는 것이고, 사람이 지정할 것은 레이어의 층수나 학습할 필터의 크기 정도이다.

2차원 이미지를 다루는 컨볼루션 레이어를 생성하는 코드

In [ ]:
import tensorflow as tf
conv1 = tf.keras.layers.Conv2D(kernel_size=(3,3), strides=(2,2),padding='valid', filters=16)

##Conv2D 레이어의 주요 인수


1. kernel_size : 필터 행렬의 크기 **(=receptive field)**, 숫자를 하나만 사용할 경우 높이와 너비를 같게 한다는 의미
2. strides : 한 스텝마다 이동하는 크기, 숫자를 하나만 적을 시 kernel_size와 동일하게 높이와 너비를 같게 한다고 취급함
3. padding : 이미지 크기를 같도록 보존하는 것으로 valid 시 비적용, same시 적용
4. filters : 필터의 갯수, VGG에서는 2배씩 증가(64->128->256->512)


##풀링 레이어(Pooling Layer)


: 이미지를 구성하는 픽셀 중 인접한 픽셀들이 비슷한 정보를 갖는 경우가 많다.    
이러한 `이미지의 크기를 줄이면서 중요한 정보만 남기기 위해 서브샘플링(subsampling)`이라는 기법을 사용하는데, 이것이 풀링 레이어이다.    
이점 : **컴퓨터 메모리의 효율적 사용**, **계산 정보의 감소에 따른 과적합 방지**    
종류 : Max 풀링 레이어, Average 풀링 레이어    
Max 풀링 레이어는 범위 최대값 만을 남기는 것, Average 풀링 레이어는 평균값을 남기는 것이다.    
컨볼루션 레이어에서는 보통 Max 풀링 레이어만을 사용

In [ ]:
pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))

#####그럼 풀사이즈랑 스트라이드랑 같게하나??

위 코드 실행시에 이미지의 너비, 높이가 절반으로 줄어듬.    
풀링 레이어의 경우에는 가중치가 존재하지 않기 때문에 학습되지 않고, 생략되기도 함.

##드롭아웃 레이어

: 과적합 방지를 위해 학습 과정에서 뉴런의 부분집합을 제거하는 것이다.    
학습 당시에는 같은 레이어의 뉴런들은 결과 값에 의해 서로 같은 영향을 받는다. 따라서 각 `뉴런의 계산 결과를 모두 더해서 나오는 결괏값은 한쪽으로 치우치게 된다.` 이를 막기 위해 학습 과정에서 `확률적으로 일부 뉴런에 대한 연결을 끊고 테스트 할 때는 정상적으로 모든 값을 포함`해서 계산한다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/drop_out.png', width=600)

In [ ]:
pool1 = tf.keras.layers.Dropout(rate=0.3)

rate : 제외할 뉴런의 비율     
drop out은 컨볼루션 레이어 에서만 학습이 가능하다

##Fashion MNIST 데이터세트에 적용하기

In [ ]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

train_X = train_X / 255.0
test_X = test_X / 255.0

###이미지 reshape

이미지의 경우 Channel을 갖기 때문에 학습 전에 데이터의 Shape를 바꿔야 함

In [ ]:
#reshape 이전
print(train_X.shape, test_X.shape)

train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

#reshape 이후
print(train_X.shape, test_X.shape)

In [ ]:
import matplotlib.pyplot as plt
# 전체 그래프의 크기를 width=10, height=10으로 지정
plt.figure(figsize=(10,10))
for c in range(16):
  # 4행 4열로 지정한 그리드에서 c+1번째의 칸에 그래프를 그린다. 1~16번째 칸을 채움
  plt.subplot(4,4,c+1)
  plt.imshow(train_X[c].reshape(28,28), cmap='gray')

plt.show()

# 훈련 데이터의 첫 번째 ~ 16 번째 까지의 라벨 프린트
print(train_Y[:16])

###모델 생성

#####Conv2D filter??

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), filters=16),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
!nvidia-smi

###학습

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(test_X, test_Y, verbose=0)

그래프를 보면 전형적인 과적합의 형태가 나온다.    
이번에 풀링과 드롭아웃 레이어를 모두 적용해 보자.

###Max Pooling, Drop out 적용

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), filters=32),
        tf.keras.layers.MaxPool2D(strides=(2,2)),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64),
        tf.keras.layers.MaxPool2D(strides=(2,2)),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dropout(rate=0.3),
        tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

구조를 보면 파라미터 개수가 Max 풀링과 드롭아웃을 적용하기 전의 6% 밖에 안되는 수준으로 감소했음을 볼 수 있다. 풀링 레이어에 의해 이미지의 크기가 감소해 flatten 단계에서 30,976에서 1,152로 감소 했기 때문이다.

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(test_X, test_Y, verbose=0)

여전히 val_loss가 증가하지만 val_accuracy는 일정한 수준에 머무르고 있다. Dense에 비해 성능이 증가했다. 성능을 더 올려보자

###퍼포먼스 높이기

컨볼루션 신경망에서 퍼포먼스를 높이는 대표적이면서 쉬운 두가지 방법은 `더많은 레이어 쌓기`와 `이미지 보강(Image Augmentation)`이다.

역사적으로 컨볼루션 레이어에서는 더 깊은 신경망을 쌓기 위한 노력이라고 해도 과언이 아니다.

####VGGNet 스타일의 Fashion MNIST 분류를 위한 모델 정의

#####? 이게 무슨 근거의 구조인가

1.   컨볼루션 중간에 드롭아웃? (원래 VGG의 특징이 이것)
2.   패딩 valid? (default임)



VGG-19에서는 (컨볼루션 레이어 2개 + 풀링 레이어 1개)패턴 2차례, (컨볼루션 레이어 4개 + 풀링 레이어 1개)패턴 3차례 반복이다.    
이 학습 데이터의 경우에는 `? 근거 :이미지가 작고 연산 능력의 한계`도 있어서 2 conv + 1 pooling 패턴을 2차례 반복하였고, `풀링 레이어 다음에 드롭아웃 레이어를 위치시켜 과적합을 방지`했으며, Flatten 레이어 다음에 3개의 Dense레이어 사이에도 드롭아웃을 배치 하였다. VGG식으로 층수를 센다면 VGG-7정도가 된다.

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), padding='same',filters=32, activation='relu'),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

####학습

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(test_X, test_Y, verbose=0)

val_loss가 증가하지 않는 훌륭한 결과를 얻었다.    
하지만 모델이 과적합되지 않았기 때문에 epoch를 늘려서 더 학습해볼 수 있다.

###이미지 보강

: 훈련 데이터에 없는 이미지를 새로 만들어내서 훈련 데이터를 보강하는 것    
뒤집기, 기울이기, 일부 확대, 평행이동 등..

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

image_generator = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.10,
    shear_range = 0.5,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    vertical_flip=False)

augment_size = 100

x_augmented = image_generator.flow(np.tile(train_X[0].reshape(28*28),100).reshape(-1,28,28,1),
                     np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]

#새롭게 생성된 이미지 표시
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for c in range(100):
  plt.subplot(10,10,c+1)
  plt.axis('off')
  plt.imshow(x_augmented[c].reshape(28,28), cmap='gray')
plt.show()

####ImageDataGenerator의 주요 인수





1. rotation_ragne
2. zoom_range
3. shear_range : 기울기 정도
4. horizontal_filp    

가로로 뒤집는 horizontal_filp은 사용하지만 Fashion MNIST는 이미지가 위아래로 반듯하게 놓여 있기 때문에 vertical_flip 옵션을 True로 설정하면 대비하지 않아도 될 경우에 대해서도 대비하게 되어 오히려 퍼포먼스가 떨어진다.

####flow() : 실제로 보강된 이미지를 생성한다.

In [ ]:
image_generator = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.10,
    shear_range = 0.5,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    vertical_flip=False)

augment_size = 30000

randidx = np.random.randint(train_X.shape[0], size=augment_size)
x_augmented = train_X[randidx].copy()
y_augmented = train_Y[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]

# 원래 데이터인 x_train에 이미지 보강된 x_augmented를 추가한다.

train_X = np.concatenate((train_X, x_augmented))
train_Y = np.concatenate((train_Y, y_augmented))

print(train_X.shape)

###보강 이미지를 기반으로 재학습

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), padding='same',filters=32, activation='relu'),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(test_X, test_Y, verbose=0)

성적은 소폭 증가 했고, 역시 아직 과적합 되지 않았기 때문에 더 학습시킬 수 있음.